## Predict Shakespeare with Cloud TPUs and Keras

## Overview

This example uses [tf.keras](https://www.tensorflow.org/guide/keras) to build a *language model* and train it on a Cloud TPU. This language model predicts the next character of text given the text so far. The trained model can generate new snippets of text that read in a similar style to the text training data.

The model trains for 10 epochs and completes in approximately 5 minutes.

This notebook is hosted on GitHub. To view it in its original repository, after opening the notebook, select **File > View on GitHub**.

## Learning objectives

In this Colab, you will learn how to:
*   Build a two-layer, forward-LSTM model.
*   Use distribution strategy to produce a `tf.keras` model that runs on TPU version and then use the standard Keras methods to train: `fit`, `predict`, and `evaluate`.
*   Use the trained model to make predictions and generate your own Shakespeare-esque play.






## Instructions

<h3>  &nbsp;&nbsp;Train on TPU&nbsp;&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a></h3>

   1. On the main menu, click Runtime and select **Change runtime type**. Set "TPU" as the hardware accelerator.
   1. Click Runtime again and select **Runtime > Run All**. You can also run the cells manually with Shift-ENTER. 

TPUs are located in Google Cloud, for optimal performance, they read data directly from Google Cloud Storage (GCS)

## Data, model, and training

In this example, you train the model on the combined works of William Shakespeare, then use the model to compose a play in the style of *The Great Bard*:

<blockquote>
Loves that led me no dumbs lack her Berjoy's face with her to-day.  
The spirits roar'd; which shames which within his powers  
	Which tied up remedies lending with occasion,  
A loud and Lancaster, stabb'd in me  
	Upon my sword for ever: 'Agripo'er, his days let me free.  
	Stop it of that word, be so: at Lear,  
	When I did profess the hour-stranger for my life,  
	When I did sink to be cried how for aught;  
	Some beds which seeks chaste senses prove burning;  
But he perforces seen in her eyes so fast;  
And _  
</blockquote>


### Download data

Download *The Complete Works of William Shakespeare* as a single text file from [Project Gutenberg](https://www.gutenberg.org/). You use snippets from this file as the *training data* for the model. The *target* snippet is offset by one character.

In [1]:
!wget --show-progress --continue -O /content/shakespeare.txt http://www.gutenberg.org/files/100/100-0.txt

--2020-08-31 14:35:57--  http://www.gutenberg.org/files/100/100-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



### Build the input dataset

We just downloaded some text. The following shows the start of the text and a random snippet so we can get a feel for the whole text.

In [2]:
!head -n5 /content/shakespeare.txt
!echo "..."
!shuf -n5 /content/shakespeare.txt

﻿Project Gutenberg’s The Complete Works of William Shakespeare, by William Shakespeare

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever.  You may copy it, give it away or re-use it under the terms
...
Stop thy unhallow’d toil, vile Montague.

      Give not this rotten orange to your friend;

And those boils did run—say so. Did not the general run then? Were not


In [3]:
import numpy as np
import tensorflow as tf
import os

import distutils
if distutils.version.LooseVersion(tf.__version__) < '2.0':
    raise Exception('This notebook is compatible with TensorFlow 2.0 or higher.')

SHAKESPEARE_TXT = '/content/shakespeare.txt'

def transform(txt):
  return np.asarray([ord(c) for c in txt if ord(c) < 255], dtype=np.int32)

def input_fn(seq_len=100, batch_size=1024):
  """Return a dataset of source and target sequences for training."""
  with tf.io.gfile.GFile(SHAKESPEARE_TXT, 'r') as f:
    txt = f.read()

  source = tf.constant(transform(txt), dtype=tf.int32)

  ds = tf.data.Dataset.from_tensor_slices(source).batch(seq_len+1, drop_remainder=True)

  def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

  BUFFER_SIZE = 10000
  ds = ds.map(split_input_target).shuffle(BUFFER_SIZE).batch(batch_size, drop_remainder=True)

  return ds.repeat()

### Build the model

The model is defined as a two-layer, forward-LSTM, the same model should work both on CPU and TPU.

Because our vocabulary size is 256, the input dimension to the Embedding layer is 256.

When specifying the arguments to the LSTM, it is important to note how the stateful argument is used. When training we will make sure that `stateful=False` because we do want to reset the state of our model between batches, but when sampling (computing predictions) from a trained model, we want `stateful=True` so that the model can retain information across the current batch and generate more interesting text.

In [4]:
EMBEDDING_DIM = 512

def lstm_model(seq_len=100, batch_size=None, stateful=True):
  """Language model: predict the next word given the current word."""
  source = tf.keras.Input(
      name='seed', shape=(seq_len,), batch_size=batch_size, dtype=tf.int32)

  embedding = tf.keras.layers.Embedding(input_dim=256, output_dim=EMBEDDING_DIM)(source)
  lstm_1 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(embedding)
  lstm_2 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(lstm_1)
  predicted_char = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='softmax'))(lstm_2)
  return tf.keras.Model(inputs=[source], outputs=[predicted_char])

### Train the model

First, we need to create a distribution strategy that can use the TPU. In this case it is TPUStrategy. You can create and compile the model inside its scope. Once that is done, future calls to the standard Keras methods `fit`, `evaluate` and `predict` use the TPU.

Again note that we train with `stateful=False` because while training, we only care about one batch at a time.

In [5]:
tf.keras.backend.clear_session()

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.experimental.TPUStrategy(resolver)

with strategy.scope():
  training_model = lstm_model(seq_len=100, stateful=False)
  training_model.compile(
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])

training_model.fit(
    input_fn(),
    steps_per_epoch=100,
    epochs=20
)
training_model.save_weights('/content/drive/My Drive/tmp/bard.h5', overwrite=True)

INFO:tensorflow:Initializing the TPU system: grpc://10.11.115.98:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.11.115.98:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Epoch 1/20
  2/100 [..............................] - ETA: 1:51 - loss: 5.1552 - sparse_categorical_accuracy: 0.0868WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0019s vs `on_train_batch_end` time: 0.0561s). Check your callbacks.


100/100 [==============================] - 9s 93ms/step - loss: 3.4739 - sparse_categorical_accuracy: 0.1531
Epoch 2/20
100/100 [==============================] - 8s 77ms/step - loss: 2.1588 - sparse_categorical_accuracy: 0.3790
Epoch 3/20
100/100 [==============================] - 8s 78ms/step - loss: 1.5927 - sparse_categorical_accuracy: 0.5229
Epoch 4/20
100/100 [==============================] - 8s 77ms/step - loss: 1.3882 - sparse_categorical_accuracy: 0.5757
Epoch 5/20
100/100 [==============================] - 8s 76ms/step - loss: 1.3038 - sparse_categorical_accuracy: 0.5966
Epoch 6/20
100/100 [==============================] - 7s 72ms/step - loss: 1.2536 - sparse_categorical_accuracy: 0.6098
Epoch 7/20
100/100 [==============================] - 8s 78ms/step - loss: 1.2213 - sparse_categorical_accuracy: 0.6180
Epoch 8/20
100/100 [==============================] - 8s 75ms/step - loss: 1.2041 - sparse_categorical_accuracy: 0.6223
Epoch 9/20
100/100 [==============================]

### Make predictions with the model

Use the trained model to make predictions and generate your own Shakespeare-esque play.
Start the model off with a *seed* sentence, then generate 250 characters from it. The model makes five predictions from the initial seed.

The predictions are done on the CPU so the batch size (5) in this case does not have to be divisible by 8.

Note that when we are doing predictions or, to be more precise, text generation, we set `stateful=True` so that the model's state is kept between batches. If stateful is false, the model state is reset between each batch, and the model will only be able to use the information from the current batch (a single character) to make a prediction.

The output of the model is a set of probabilities for the next character (given the input so far). To build a paragraph, we predict one character at a time and sample a character (based on the probabilities provided by the model). For example, if the input character is "o" and the output probabilities are "p" (0.65), "t" (0.30), others characters (0.05), then we allow our model to generate text other than just "Ophelia" and "Othello."

In [6]:
BATCH_SIZE = 5
PREDICT_LEN = 250

# Keras requires the batch size be specified ahead of time for stateful models.
# We use a sequence length of 1, as we will be feeding in one character at a 
# time and predicting the next character.
prediction_model = lstm_model(seq_len=1, batch_size=BATCH_SIZE, stateful=True)
prediction_model.load_weights('/content/drive/My Drive/tmp/bard.h5')

# We seed the model with our initial string, copied BATCH_SIZE times

seed_txt = 'Looks it not like the king?  Verily, we must go! '
seed = transform(seed_txt)
seed = np.repeat(np.expand_dims(seed, 0), BATCH_SIZE, axis=0)

# First, run the seed forward to prime the state of the model.
prediction_model.reset_states()
for i in range(len(seed_txt) - 1):
  prediction_model.predict(seed[:, i:i + 1])

# Now we can accumulate predictions!
predictions = [seed[:, -1:]]
for i in range(PREDICT_LEN):
  last_word = predictions[-1]
  next_probits = prediction_model.predict(last_word)[:, 0, :]
  
  # sample from our output distribution
  next_idx = [
      np.random.choice(256, p=next_probits[i])
      for i in range(BATCH_SIZE)
  ]
  predictions.append(np.asarray(next_idx, dtype=np.int32))
  

for i in range(BATCH_SIZE):
  print('PREDICTION %d\n\n' % i)
  p = [predictions[j][i] for j in range(PREDICT_LEN)]
  generated = ''.join([chr(c) for c in p])  # Convert back to text
  print(generated)
  print()
  assert len(generated) == PREDICT_LEN, 'Generated text too short'

PREDICTION 0


 Ay
a deud in our despites, or have scourge, not like him? This
instant and to bear us; to paint the man his lightning, my sweet movs.

WILLIAMS.
Come, bear me hence, my litters.

FRANCE.
What is this?

VARRO.
Unbuckled. I know he welcome 

PREDICTION 1


 blue, send no more?

HAMLET.
Why, Jessica, my lord?

CHARMIAN.
A danger piece of his; for I think this sleep and reinous.

SOOTHSAYER.
Ill beard my chiefes,
A proper king, winner nature denied
In displeasure.

ANTIGONUS.
But my breakfa

PREDICTION 2


 adieu?
  TALBOT. Why?
  BEROWNE. Not till I have derisions.
  BOYET. Why, Gloucester, sit one; I am assured.
  PRINCESS OF FRANCE. True, ne'er personally. Go serve, they are now and night.
  QUEEN MARGARET. And so am, stand by the cacklings,
 

PREDICTION 3


 by Saint Macbeth, why should I not name his
rate?

PRINCE.
[_To Carries, man, Charmian._]
Lest thanked sorrow, madam:
Where? or both, hence! Mend ye, in dust.

LADY MACBETH.
Not so, here a devil:
God you come not a